In [1]:
import pandas as pd
import numpy as np
from collections import Counter
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GroupShuffleSplit
import uuid
from tqdm import tqdm

In [2]:
measurement_meds_test = pd.read_csv('testing_data/measurement_meds_test.csv')
measurement_observation_test =  pd.read_csv('testing_data/measurement_observation_test.csv')
observation_test = pd.read_csv('testing_data/observation_test.csv')
person_demographics_episode_test = pd.read_csv('testing_data/person_demographics_episode_test.csv')
proceduresoccurrences_test = pd.read_csv('testing_data/proceduresoccurrences_test.csv')
sepsis_label_test = pd.read_csv('testing_data/SepsisLabel_test.csv')
devices_test = pd.read_csv('testing_data/devices_test.csv')
drugsexposure_test = pd.read_csv('testing_data/drugsexposure_test.csv')
measurement_lab_test = pd.read_csv('testing_data/measurement_lab_test.csv')

In [3]:
measurement_meds_train = pd.read_csv('training_data/measurement_meds_train.csv').drop_duplicates()
measurement_observation_train =  pd.read_csv('training_data/measurement_observation_train.csv').drop_duplicates()
observation_train = pd.read_csv('training_data/observation_train.csv').drop_duplicates()
person_demographics_episode_train = pd.read_csv('training_data/person_demographics_episode_train.csv').drop_duplicates()
proceduresoccurrences_train = pd.read_csv('training_data/proceduresoccurrences_train.csv').drop_duplicates()
sepsis_label_train = pd.read_csv('training_data/SepsisLabel_train.csv').drop_duplicates()
devices_train = pd.read_csv('training_data/devices_train.csv').drop_duplicates()
drugsexposure_train = pd.read_csv('training_data/drugsexposure_train.csv').drop_duplicates()
measurement_lab_train = pd.read_csv('training_data/measurement_lab_train.csv').drop_duplicates()

train_dfs = [measurement_meds_train, measurement_observation_train, observation_train, proceduresoccurrences_train, sepsis_label_train, devices_train, drugsexposure_train, measurement_lab_train]

In [4]:
def uid_maker(df_list: list):
    for j, df in enumerate(df_list):
        # print(df.columns)
        datetime_index = np.argmax([i.find('datetime') for i in df.columns])
        date_column = df.columns[datetime_index]
        personid_index = np.argmax([i.find('person_id') for i in df.columns])
        personid_column = df.columns[personid_index]
        # print(f'date col: {date_column} personid: {personid_column}')
        df['uid'] = df[date_column].astype(str) + df[personid_column].astype(str)
        if j != 4:
            df.drop(columns=[date_column,personid_column],inplace=True)

uid_maker(train_dfs)

In [106]:
print(len(observation_train))
print(len(sepsis_label_train))
print(len(drugsexposure_train))
print(len(measurement_lab_train))
print(len(measurement_meds_train))
print(len(proceduresoccurrences_train))
print(len(measurement_observation_train))
print(len(person_demographics_episode_train))
print(len(devices_train))

3807
331653
184780
69307
257749
771214
50553
3391
750878


In [5]:
print(len(observation_train))
print(len(sepsis_label_train))
print(len(drugsexposure_train))
print(len(measurement_lab_train))
print(len(measurement_meds_train))
print(len(proceduresoccurrences_train))
print(len(measurement_observation_train))
print(len(person_demographics_episode_train))
print(len(devices_train))

3807
331639
182929
69307
257749
771214
50553
3391
750878


In [6]:
drugsexposure_train_2 = pd.DataFrame(columns = ['uid', 'drugs', 'routes'])
for x in tqdm(np.unique(drugsexposure_train['uid'])):
    drugs = drugsexposure_train[drugsexposure_train['uid']==x]['drug_concept_id'].to_list()
    drugs.sort()
    try:
        route = drugsexposure_train[drugsexposure_train['uid']==x]['route_concept_id'].to_list()
        route = list(set(route))
        route = [str(i) for i in route]
        route.sort()
    except:
        route = drugsexposure_train[drugsexposure_train['uid']==x]['route_concept_id'].to_list()
        route = list(set(route))
    drugsexposure_train_2.loc[len(drugsexposure_train_2)]= [x,drugs,route]

100%|██████████| 150407/150407 [1:14:55<00:00, 33.46it/s]


In [7]:
drugsexposure_train_2.to_csv('processed_drug_exposure_train.csv')

In [ ]:
devices_train.drop(columns=['visit_occurrence_id'],inplace=True)

In [39]:
current_variables = pd.merge(left=sepsis_label_train, right= drugsexposure_train_2, how= 'left', on='uid')

In [40]:
current_variables = pd.merge(left=current_variables, right= devices_train, how= 'left', on='uid')

In [52]:
measurement_lab_train_2 = measurement_lab_train
print(len(measurement_lab_train))
measurement_lab_train_2.drop(columns=['visit_occurrence_id'],inplace=True)
print(f'one column gone {len(measurement_lab_train_2)}')
measurement_lab_train_2.drop_duplicates()
print(f'dupes gone {len(measurement_lab_train_2)}')

69307
one column gone 69307
dupes gone 69307


In [73]:
pd.set_option('display.max_columns', None)
measurement_lab_train_2[measurement_lab_train_2['uid']=='2024-07-01 16:00:00131047132']

,Base excess in Venous blood by calculation,Base excess in Arterial blood by calculation,Phosphate [Moles/volume] in Serum or Plasma,Potassium [Moles/volume] in Blood,Bilirubin.total [Moles/volume] in Serum or Plasma,Neutrophil Ab [Units/volume] in Serum,Bicarbonate [Moles/volume] in Arterial blood,Hematocrit [Volume Fraction] of Blood,Glucose [Moles/volume] in Serum or Plasma,Calcium [Moles/volume] in Serum or Plasma,Chloride [Moles/volume] in Blood,Sodium [Moles/volume] in Serum or Plasma,C reactive protein [Mass/volume] in Serum or Plasma,Carbon dioxide [Partial pressure] in Venous blood,Oxygen [Partial pressure] in Venous blood,Albumin [Mass/volume] in Serum or Plasma,Bicarbonate [Moles/volume] in Venous blood,Oxygen [Partial pressure] in Arterial blood,Carbon dioxide [Partial pressure] in Arterial blood,Interleukin 6 [Mass/volume] in Body fluid,Magnesium [Moles/volume] in Blood,Prothrombin time (PT),Procalcitonin [Mass/volume] in Serum or Plasma,Lactate [Moles/volume] in Blood,Creatinine [Mass/volume] in Blood,Fibrinogen measurement,Bilirubin measurement,Partial thromboplastin time,activated,Total white blood count,Platelet count,White blood cell count,Blood venous pH,D-dimer level,Blood arterial pH,Hemoglobin [Moles/volume] in Blood,Ionised calcium measurement,uid
318,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-07-01 16:00:00131047132
35546,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-07-01 16:00:00131047132


In [54]:
Counter(measurement_lab_train_2['uid'])

Counter({'2024-07-01 16:00:00131047132': 2,
         '2024-07-19 12:00:00131047132': 2,
         '2024-07-19 13:00:00131047132': 2,
         '2024-09-02 16:00:00179116795': 2,
         '2024-09-02 17:00:00179116795': 2,
         '2024-09-30 09:00:00179116795': 2,
         '2023-12-06 17:00:00523870286': 1,
         '2023-12-06 18:00:00523870286': 1,
         '2023-12-07 08:00:00523870286': 1,
         '2023-12-07 09:00:00523870286': 1,
         '2023-12-08 23:00:00523870286': 1,
         '2021-01-04 08:00:002097595915': 1,
         '2021-01-04 09:00:002097595915': 1,
         '2021-01-05 20:00:002097595915': 1,
         '2021-01-05 21:00:002097595915': 1,
         '2021-01-07 07:00:002097595915': 1,
         '2021-01-07 08:00:002097595915': 1,
         '2021-01-07 09:00:002097595915': 1,
         '2021-01-08 05:00:002097595915': 1,
         '2021-01-08 06:00:002097595915': 1,
         '2021-01-12 10:00:002097595915': 1,
         '2021-01-12 11:00:002097595915': 1,
         '2021-01-12 

In [66]:
person_ids = np.unique(measurement_meds_train['person_id'])

In [67]:
measurement_meds_train[measurement_meds_train['person_id']==person_ids[9]]

,visit_occurrence_id,person_id,measurement_datetime,Systolic blood pressure,Diastolic blood pressure,Body temperature,Respiratory rate,Heart rate,Measurement of oxygen saturation at periphery,Oxygen/Gas total [Pure volume fraction] Inhaled gas,uid
71027,644381400,5890051,2023-06-12 14:00:00,NaN,NaN,35.8,NaN,NaN,NaN,NaN,2023-06-12 14:00:005890051
71028,644381400,5890051,2023-06-12 16:00:00,NaN,NaN,36.1,NaN,NaN,NaN,NaN,2023-06-12 16:00:005890051
71029,644381400,5890051,2023-06-12 18:00:00,NaN,NaN,36.0,NaN,NaN,NaN,NaN,2023-06-12 18:00:005890051
71030,644381400,5890051,2023-06-12 19:00:00,NaN,NaN,36.5,NaN,NaN,NaN,NaN,2023-06-12 19:00:005890051
71031,644381400,5890051,2023-06-12 21:00:00,NaN,NaN,36.2,NaN,NaN,NaN,NaN,2023-06-12 21:00:005890051
71032,644381400,5890051,2023-06-12 23:00:00,NaN,NaN,36.7,NaN,NaN,NaN,NaN,2023-06-12 23:00:005890051
71033,644381400,5890051,2023-06-13 01:00:00,NaN,NaN,36.0,NaN,NaN,NaN,NaN,2023-06-13 01:00:005890051
71034,644381400,5890051,2023-06-13 06:00:00,NaN,NaN,36.0,NaN,NaN,NaN,NaN,2023-06-13 06:00:005890051
71035,644381400,5890051,2023-06-13 09:00:00,NaN,NaN,35.9,NaN,NaN,NaN,NaN,2023-06-13 09:00:005890051
248060,2084208524,5890051,2024-09-12 17:00:00,NaN,NaN,35.9,20.614035,106.586205,99.421875,NaN,2024-09-12 17:00:005890051


In [68]:
len(np.unique(sepsis_label_train[sepsis_label_train['SepsisLabel']==1]['person_id']))

97

In [7]:
len(np.unique(sepsis_label_train[sepsis_label_train['SepsisLabel']==0]['person_id']))

2649

In [8]:
train = np.unique(sepsis_label_train['person_id'])
test = np.unique(sepsis_label_test['person_id'])

for x in test:
    if x in train:
        print("weird")

weird
weird
weird
weird
weird
weird
weird
weird
weird
weird
weird
weird
weird
weird
weird
weird
weird
weird


In [11]:
Counter(proceduresoccurrences_train['uid'])

Counter({'2024-08-20 12:00:001576271591': 2,
         '2024-08-20 13:00:001576271591': 2,
         '2024-08-20 14:00:001576271591': 2,
         '2024-08-20 15:00:001576271591': 2,
         '2024-08-20 16:00:001576271591': 2,
         '2024-08-20 17:00:001576271591': 2,
         '2024-08-20 18:00:001576271591': 2,
         '2024-08-20 19:00:001576271591': 2,
         '2024-08-20 20:00:001576271591': 2,
         '2024-08-20 21:00:001576271591': 2,
         '2024-08-20 22:00:001576271591': 2,
         '2024-08-20 23:00:001576271591': 2,
         '2024-08-21 00:00:001576271591': 2,
         '2024-08-21 01:00:001576271591': 2,
         '2024-08-21 02:00:001576271591': 2,
         '2024-08-21 03:00:001576271591': 2,
         '2024-08-21 04:00:001576271591': 2,
         '2024-08-21 05:00:001576271591': 2,
         '2024-08-21 06:00:001576271591': 2,
         '2024-08-21 07:00:001576271591': 2,
         '2024-08-21 08:00:001576271591': 2,
         '2024-08-21 09:00:001576271591': 2,
         '

In [12]:
proceduresoccurrences_train[proceduresoccurrences_train['uid']=='2024-08-20 12:00:001576271591']

,visit_occurrence_id,procedure,uid
339416,649576332,Cannulation,2024-08-20 12:00:001576271591
411005,795368156,Cannulation,2024-08-20 12:00:001576271591


In [15]:
proceduresoccurrences_train_2 = pd.read_csv('training_data/proceduresoccurrences_train.csv').drop_duplicates()

In [24]:
proceduresoccurrences_train_2.sample(5)

,visit_occurrence_id,procedure_datetime_hourly,person_id,procedure
645346,1162738146,2019-11-14 08:00:00,402750444,Non-invasive ventilation
147152,284394132,2024-10-21 09:00:00,1383234881,Cannulation
87252,169508830,2019-07-07 09:00:00,563540406,Cannulation
50209,109249733,2023-12-15 11:00:00,454562338,Non-invasive ventilation
185866,355912728,2021-02-03 14:00:00,2129787639,Cannulation


In [32]:
proceduresoccurrences_train_2[(proceduresoccurrences_train_2['person_id']==1576271591) & (proceduresoccurrences_train_2['procedure_datetime_hourly'] == '2024-08-21 03:00:00')]

,visit_occurrence_id,procedure_datetime_hourly,person_id,procedure
339431,649576332,2024-08-21 03:00:00,1576271591,Cannulation
411020,795368156,2024-08-21 03:00:00,1576271591,Cannulation


In [42]:
drugsexposure_train.sample(6)

,visit_occurrence_id,drug_concept_id,route_concept_id,uid
7959,100527343,milrinone,Intravenous,2020-02-24 12:00:001374751557
80163,961570967,voriconazole,Intravenous,2021-09-02 06:00:00183894266
130712,1519452198,epinephrine,Intravenous,2022-06-08 12:00:001882124297
164665,1911747417,norepinephrine,Intravenous,2020-12-25 06:00:00450864994
152131,1750127967,milrinone,Intravenous,2024-06-18 10:00:00387523285
78627,948898399,isavuconazole,Intravenous,2023-10-27 07:00:00109332931


In [ ]:
devices_train.sample(5)

,device,uid
654646,Nasogastric/orogastric tube stylet,2024-07-17 03:00:00200935776
57270,Nasogastric/orogastric tube stylet,2022-12-08 17:00:001698655074
652493,Central venous catheter,2022-08-20 01:00:001551731969
649187,Central venous catheter,2024-07-20 23:00:001952568272
505008,Urinary catheter,2023-11-10 13:00:00109332931


In [121]:
observation_train.columns

Index(['visit_occurrence_id', 'person_id', 'observation_datetime',
       'observation_concept_id', 'observation_concept_name', 'valuefilled'],
      dtype='object')

In [73]:
measurement_observation_train.sample(6)

,visit_occurrence_id,person_id,measurement_datetime,Left pupil Diameter Auto,Right pupil Diameter Auto,Glasgow coma scale,Capillary refill [Time],Pulse,Arterial pulse pressure,Right pupil Pupillary response,Left pupil Pupillary response,uid
39084,1682818919,1939094204,2022-03-08 13:00:00,3.0,3.0,15.0,NaN,NaN,NaN,Normal,Normal,2022-03-08 13:00:001939094204
11542,554200535,1201581490,2020-07-26 19:00:00,3.5,3.5,15.0,Normal capillary filling,NaN,Present,Normal,Normal,2020-07-26 19:00:001201581490
19008,866895186,203008612,2022-06-12 05:00:00,NaN,NaN,15.0,NaN,NaN,NaN,NaN,NaN,2022-06-12 05:00:00203008612
30163,1307478059,505631993,2023-03-03 04:00:00,3.0,3.0,NaN,NaN,NaN,NaN,Normal,Normal,2023-03-03 04:00:00505631993
8222,373578703,1376769416,2019-09-14 11:00:00,NaN,NaN,NaN,Normal capillary filling,Present,Present,NaN,NaN,2019-09-14 11:00:001376769416
41963,1826807599,1437492736,2023-02-10 21:00:00,4.0,4.0,15.0,NaN,NaN,NaN,Normal,Normal,2023-02-10 21:00:001437492736


In [74]:
measurement_lab_train.sample(5)

,visit_occurrence_id,person_id,measurement_datetime,Base excess in Venous blood by calculation,Base excess in Arterial blood by calculation,Phosphate [Moles/volume] in Serum or Plasma,Potassium [Moles/volume] in Blood,Bilirubin.total [Moles/volume] in Serum or Plasma,Neutrophil Ab [Units/volume] in Serum,Bicarbonate [Moles/volume] in Arterial blood,...,activated,Total white blood count,Platelet count,White blood cell count,Blood venous pH,D-dimer level,Blood arterial pH,Hemoglobin [Moles/volume] in Blood,Ionised calcium measurement,uid
36902,1176219849,1587833691,2021-07-07 07:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021-07-07 07:00:001587833691
4721,141752566,650167967,2020-07-17 08:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,9.5,1070.0,NaN,NaN,NaN,NaN,9.7,NaN,NaN,2020-07-17 08:00:00650167967
7148,207491975,491923945,2020-01-04 06:00:00,7.4,NaN,1.63,3.20,NaN,6.8,NaN,...,17.7,127.0,NaN,7.42,NaN,NaN,13.8,1.27,NaN,2020-01-04 06:00:00491923945
6457,191631462,176946437,2019-10-14 10:00:00,NaN,NaN,NaN,4.35,2.9,2.4,NaN,...,3.0,79.0,NaN,NaN,NaN,NaN,9.8,NaN,NaN,2019-10-14 10:00:00176946437
28124,939989098,392831439,2021-07-06 18:00:00,-16.3,NaN,1.94,4.10,NaN,13.6,NaN,...,17.4,258.0,NaN,7.23,NaN,NaN,11.3,1.11,NaN,2021-07-06 18:00:00392831439


In [47]:
Counter(measurement_lab_train['uid'])

Counter({'2024-07-01 16:00:00131047132': 2,
         '2024-07-19 12:00:00131047132': 2,
         '2024-07-19 13:00:00131047132': 2,
         '2024-09-02 16:00:00179116795': 2,
         '2024-09-02 17:00:00179116795': 2,
         '2024-09-30 09:00:00179116795': 2,
         '2023-12-06 17:00:00523870286': 1,
         '2023-12-06 18:00:00523870286': 1,
         '2023-12-07 08:00:00523870286': 1,
         '2023-12-07 09:00:00523870286': 1,
         '2023-12-08 23:00:00523870286': 1,
         '2021-01-04 08:00:002097595915': 1,
         '2021-01-04 09:00:002097595915': 1,
         '2021-01-05 20:00:002097595915': 1,
         '2021-01-05 21:00:002097595915': 1,
         '2021-01-07 07:00:002097595915': 1,
         '2021-01-07 08:00:002097595915': 1,
         '2021-01-07 09:00:002097595915': 1,
         '2021-01-08 05:00:002097595915': 1,
         '2021-01-08 06:00:002097595915': 1,
         '2021-01-12 10:00:002097595915': 1,
         '2021-01-12 11:00:002097595915': 1,
         '2021-01-12 

In [43]:
sepsis_label_train.sample(5)

,person_id,measurement_datetime,SepsisLabel,uid
70256,1200893874,2023-05-09 13:00:00,0,2023-05-09 13:00:001200893874
130951,325012955,2021-10-25 19:00:00,0,2021-10-25 19:00:00325012955
25446,2131173859,2020-01-08 19:00:00,0,2020-01-08 19:00:002131173859
52352,1970886618,2021-10-01 14:00:00,0,2021-10-01 14:00:001970886618
166286,1943755383,2023-04-18 23:00:00,0,2023-04-18 23:00:001943755383


In [30]:
sepsis_label_train[(sepsis_label_train['person_id']==183894266) & (sepsis_label_train['measurement_datetime']== '2021-08-19 18:00:00')]

,person_id,measurement_datetime,SepsisLabel,uid
299810,183894266,2021-08-19 18:00:00,0,1838942662021-08-19 18:00:00


In [18]:
sepsis_label_train['uid'] = sepsis_label_train['person_id'].astype(str) + sepsis_label_train['measurement_datetime'].astype(str)

In [20]:
devices_train['uid'] = devices_train['person_id'].astype(str) + devices_train['device_datetime_hourly'].astype(str)

In [36]:
a = sepsis_label_train[(sepsis_label_train['person_id']==183894266) & (sepsis_label_train['measurement_datetime']== '2021-08-19 18:00:00')]['uid'].item()
b = devices_train[(devices_train['person_id']==183894266) & (devices_train['device_datetime_hourly'] == '2021-08-19 18:00:00')]['uid'].item()
a == b

True